In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import tarfile
import torchvision.utils
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split

from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from models import *
import torchvision.transforms as transforms
from torch.utils.data.dataloader import DataLoader
import gc
from utils import * 
import time

In [2]:
# defining the resizing of the images
transform = transforms.Compose([transforms.Resize((100, 100)), transforms.ToTensor()])

In [3]:
# the path of the original data, the augmented data and the models
data_dir = 'D:/gende recog adv ex/data_perc'
original_data_dir = './input/gender-classification-dataset'
folder_path1 = 'D:/gende recog adv ex/final networks'
folder_path = 'D:/gende recog adv ex'

In [4]:
# reading the test data set and the augmeted data set.
dataset_test = ImageFolder(original_data_dir+"/Validation", transform = transform)
dataset_explain = ImageFolder(data_dir, transform=transform)
attacker_data_loader = DataLoader(dataset_explain, 1)

In [5]:
len(dataset_explain)

27730

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
device

device(type='cuda')

In [7]:
# reading the provider model
model = GenCnnModel()
model = to_device(GenCnnModel(), device)
model.load_state_dict(torch.load(folder_path1+'model_main_new.pth'))

<All keys matched successfully>

In [9]:
# querying the labels from the provider model
start =int(round(time.time()*1000))
new_classes = create_classes_val(attacker_data_loader, model,device)
end =int(round(time.time()*1000))
# torch.cuda.synchronize()
time_elapsed = int(round(time.time()*1000)) -start
# print(start.elapsed_time(end))
print ('test time elapsed {}ms'.format(time_elapsed))
print ('test time elapsed {}s'.format(time_elapsed/1000))

C:\Users\012709558\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


test time elapsed 280671ms
test time elapsed 280.671s


In [10]:
time_per_image = time_elapsed/len(dataset_explain)
print ('test time per image elapsed {} ms'.format(time_per_image))
print ('test time per image elapsed {} s'.format(time_per_image/1000))

test time per image elapsed 10.121565091958168 ms
test time per image elapsed 0.010121565091958168 s


In [11]:
len(new_classes)

27730

In [12]:

# inserting the labels into the data loader.
start =int(round(time.time()*1000))
dataset_explain.samples = [(x ,new_classes[i])  for i, (x, y) in enumerate(dataset_explain.samples)]
end =int(round(time.time()*1000))
# torch.cuda.synchronize()
time_elapsed = int(round(time.time()*1000)) -start
# print(start.elapsed_time(end))
print ('test time elapsed {}ms'.format(time_elapsed))
print ('test time elapsed {}s'.format(time_elapsed/1000))

test time elapsed 4ms
test time elapsed 0.004s


In [13]:
# dataset_explain.samples

In [14]:
s = 0
for _, p in model.named_parameters():
    if p.requires_grad:
        s+= p.numel()
print(s)

45109666


In [15]:
size = int(len(dataset_explain)*90/100)

In [16]:
# spliting the augmented data into training and validation
train_size = size
val_size = len(dataset_explain) - train_size

data_train, data_val = random_split(dataset_explain, [train_size, val_size])
len(data_train), len(data_val)

(24957, 2773)

In [17]:
batch_size = 16

In [18]:
# data loaders to train and evaluate the surrogate models
train_dl = DataLoader(data_train, batch_size, shuffle = True)
val_dl = DataLoader(data_val, batch_size*2)
test_dl = DataLoader(dataset_test, batch_size*2)
steel_loader = DeviceDataLoader(train_dl, device)
val_loader = DeviceDataLoader(val_dl, device)
test_loader = DeviceDataLoader(test_dl, device)

In [19]:
num_epochs = 10
opt_func = torch.optim.Adam
lr = 0.00001

In [21]:
# evaluation of the provider model
start =int(round(time.time()*1000))
acc = evaluate(model, test_loader)
end =int(round(time.time()*1000))
# torch.cuda.synchronize()
time_elapsed = int(round(time.time()*1000)) -start
# print(start.elapsed_time(end))
print ('test time elapsed {}ms'.format(time_elapsed))
print ('test time elapsed {}s'.format(time_elapsed/1000))
print(acc)

test time elapsed 65418ms
test time elapsed 65.418s
{'val_loss': 0.08898299187421799, 'val_acc': 0.9663527607917786}


In [22]:
# deletingt he provider model to clear the RAM
del model
gc.collect()
torch.cuda.empty_cache()

In [23]:
# initializing surrogate model 2
surrogate2 = GenCnnModel()
surrogate2 = to_device(GenCnnModel(), device)

In [ ]:
# the training of surrogate model 2
start =int(round(time.time()*1000))
history = fit(num_epochs, lr, surrogate2, steel_loader, val_loader, opt_func)
end =int(round(time.time()*1000))
# torch.cuda.synchronize()
time_elapsed = int(round(time.time()*1000)) -start
# print(start.elapsed_time(end))
print ('test time elapsed {}ms'.format(time_elapsed))
print ('test time elapsed {}s'.format(time_elapsed/1000))

Epoch [0], train_loss: 0.6820, val_loss: 0.6109, val_acc: 0.6625
Epoch [1], train_loss: 0.4901, val_loss: 0.3438, val_acc: 0.8434


In [25]:
# evaluation of surrogate models 2
start =int(round(time.time()*1000))
acc = evaluate(surrogate2, test_loader)
end =int(round(time.time()*1000))
# torch.cuda.synchronize()
time_elapsed = int(round(time.time()*1000)) -start
# print(start.elapsed_time(end))
print ('test time elapsed {}ms'.format(time_elapsed))
print ('test time elapsed {}s'.format(time_elapsed/1000))
print(acc)

test time elapsed 18281ms
test time elapsed 18.281s
{'val_loss': 0.2908066511154175, 'val_acc': 0.9148972630500793}


In [26]:
time_per_image = time_elapsed/len(dataset_explain)
print ('test time per image elapsed {} ms'.format(time_per_image))
print ('test time per image elapsed {} s'.format(time_per_image/1000))

test time per image elapsed 0.6592499098449333 ms
test time per image elapsed 0.0006592499098449332 s


In [27]:
# save te Surrogate model 2
torch.save(surrogate2.state_dict(), folder_path+'/networks_final/Surrogate_2.pth')

In [28]:
# delete surrogate2 to clear the RAM
del surrogate2
gc.collect()
torch.cuda.empty_cache()

In [29]:
# evaluate(model, test_loader)

In [30]:
# initializing surrogate model 1
Surrogate1 = exp_GenCnnModel()
Surrogate1 = to_device(exp_GenCnnModel(), device)

In [31]:
# the training of surrogate model 2
start =int(round(time.time()*1000))
history = fit(num_epochs, lr, Surrogate1, steel_loader, val_loader, opt_func)
end =int(round(time.time()*1000))
# torch.cuda.synchronize()
time_elapsed = int(round(time.time()*1000)) -start
# print(start.elapsed_time(end))
print ('test time elapsed {}ms'.format(time_elapsed))
print ('test time elapsed {}s'.format(time_elapsed/1000))

Epoch [0], train_loss: 0.6669, val_loss: 0.5300, val_acc: 0.7475
Epoch [1], train_loss: 0.4051, val_loss: 0.2821, val_acc: 0.8741
Epoch [2], train_loss: 0.2840, val_loss: 0.2576, val_acc: 0.8869
Epoch [3], train_loss: 0.2460, val_loss: 0.2294, val_acc: 0.8940
Epoch [4], train_loss: 0.2141, val_loss: 0.1977, val_acc: 0.9107
Epoch [5], train_loss: 0.1945, val_loss: 0.1888, val_acc: 0.9203
Epoch [6], train_loss: 0.1780, val_loss: 0.1675, val_acc: 0.9253
Epoch [7], train_loss: 0.1687, val_loss: 0.1603, val_acc: 0.9289
Epoch [8], train_loss: 0.1559, val_loss: 0.1392, val_acc: 0.9371
Epoch [9], train_loss: 0.1492, val_loss: 0.1488, val_acc: 0.9368
test time elapsed 1267966ms
test time elapsed 1267.966s


In [32]:
# evaluating surrogate model 1
start =int(round(time.time()*1000))
acc = evaluate(Surrogate1, test_loader)
end =int(round(time.time()*1000))
# torch.cuda.synchronize()
time_elapsed = int(round(time.time()*1000)) -start
# print(start.elapsed_time(end))
print ('test time elapsed {}ms'.format(time_elapsed))
print ('test time elapsed {}s'.format(time_elapsed/1000))
print(acc)

test time elapsed 23888ms
test time elapsed 23.888s
{'val_loss': 0.21679826080799103, 'val_acc': 0.9273972511291504}


In [33]:
time_per_image = time_elapsed/len(dataset_explain)
print ('test time per image elapsed {} ms'.format(time_per_image))
print ('test time per image elapsed {} s'.format(time_per_image/1000))

test time per image elapsed 0.8614496934727731 ms
test time per image elapsed 0.0008614496934727732 s


In [34]:
# save te Surrogate model 1
torch.save(Surrogate1.state_dict(), folder_path+'/networks_final/Surrogate_1.pth')

In [35]:
# delete surrogate model 1
del explainable_model1
gc.collect()
torch.cuda.empty_cache()

In [36]:
# initializing surrogate model 3
Surrogate3 = exp_GenCnnModel2()
Surrogate3 = to_device(exp_GenCnnModel2(), device) 

In [37]:
#train Surrogate3
start =int(round(time.time()*1000))
history = fit(num_epochs, lr, Surrogate3, steel_loader, val_loader, opt_func)
end =int(round(time.time()*1000))
# torch.cuda.synchronize()
time_elapsed = int(round(time.time()*1000)) -start
# print(start.elapsed_time(end))
print ('test time elapsed {}ms'.format(time_elapsed))
print ('test time elapsed {}s'.format(time_elapsed/1000))

Epoch [0], train_loss: 0.5375, val_loss: 0.3919, val_acc: 0.8202
Epoch [1], train_loss: 0.3396, val_loss: 0.2978, val_acc: 0.8619
Epoch [2], train_loss: 0.2797, val_loss: 0.2624, val_acc: 0.8759
Epoch [3], train_loss: 0.2487, val_loss: 0.2285, val_acc: 0.8960
Epoch [4], train_loss: 0.2304, val_loss: 0.2165, val_acc: 0.9028
Epoch [5], train_loss: 0.2150, val_loss: 0.1970, val_acc: 0.9142
Epoch [6], train_loss: 0.2032, val_loss: 0.1927, val_acc: 0.9127
Epoch [7], train_loss: 0.1967, val_loss: 0.1798, val_acc: 0.9206
Epoch [8], train_loss: 0.1880, val_loss: 0.1857, val_acc: 0.9142
Epoch [9], train_loss: 0.1819, val_loss: 0.1810, val_acc: 0.9215
test time elapsed 1334848ms
test time elapsed 1334.848s


In [38]:
# save Surrogate3
torch.save(Surrogate3.state_dict(), folder_path+'/networks_final/Surrogate_3.pth')

In [39]:
#evaluate Surrogate3
start =int(round(time.time()*1000))
acc = evaluate(Surrogate3, steel_loader)
end =int(round(time.time()*1000))
# torch.cuda.synchronize()
time_elapsed = int(round(time.time()*1000)) -start
# print(start.elapsed_time(end))
print ('test time elapsed {}ms'.format(time_elapsed))
print ('test time elapsed {}s'.format(time_elapsed/1000))
print(acc)

test time elapsed 49394ms
test time elapsed 49.394s
{'val_loss': 0.18225906789302826, 'val_acc': 0.9179302453994751}


In [40]:
time_per_image = time_elapsed/len(dataset_explain)
print ('test time per image elapsed {} ms'.format(time_per_image))
print ('test time per image elapsed {} s'.format(time_per_image/1000))

test time per image elapsed 1.781247746123332 ms
test time per image elapsed 0.001781247746123332 s


In [41]:
# delete Surrogate3
del Surrogate3
gc.collect()
torch.cuda.empty_cache()

In [42]:
# initializing, training and saving surrogate model 4
Surrogate4 = exp_GenCnnModel3()
Surrogate4 = to_device(exp_GenCnnModel3(), device)
start =int(round(time.time()*1000))
history = fit(num_epochs, lr, Surrogate4, steel_loader, val_loader, opt_func)
end =int(round(time.time()*1000))
time_elapsed = int(round(time.time()*1000)) -start
print ('test time elapsed {}ms'.format(time_elapsed))
print ('test time elapsed {}s'.format(time_elapsed/1000))
torch.save(Surrogate4.state_dict(), folder_path+'/networks_final/Surrogate_4.pth')

Epoch [0], train_loss: 0.6756, val_loss: 0.5916, val_acc: 0.6847
Epoch [1], train_loss: 0.4446, val_loss: 0.3245, val_acc: 0.8524
Epoch [2], train_loss: 0.3116, val_loss: 0.2713, val_acc: 0.8777
Epoch [3], train_loss: 0.2585, val_loss: 0.2491, val_acc: 0.8874
Epoch [4], train_loss: 0.2297, val_loss: 0.1983, val_acc: 0.9102
Epoch [5], train_loss: 0.2056, val_loss: 0.1752, val_acc: 0.9210
Epoch [6], train_loss: 0.1887, val_loss: 0.1692, val_acc: 0.9239
Epoch [7], train_loss: 0.1722, val_loss: 0.1550, val_acc: 0.9343
Epoch [8], train_loss: 0.1631, val_loss: 0.1567, val_acc: 0.9314
Epoch [9], train_loss: 0.1526, val_loss: 0.1403, val_acc: 0.9397
test time elapsed 1268510ms
test time elapsed 1268.51s


In [43]:

# evaluating Surrogate 4
start =int(round(time.time()*1000))
acc = evaluate(Surrogate4, steel_loader)
end =int(round(time.time()*1000))
# torch.cuda.synchronize()
time_elapsed = int(round(time.time()*1000)) -start
# print(start.elapsed_time(end))
print ('test time elapsed {}ms'.format(time_elapsed))
print ('test time elapsed {}s'.format(time_elapsed/1000))
print(acc)

test time elapsed 55815ms
test time elapsed 55.815s
{'val_loss': 0.13803766667842865, 'val_acc': 0.9407051205635071}


In [44]:
time_per_image = time_elapsed/len(dataset_explain)
print ('test time per image elapsed {} ms'.format(time_per_image))
print ('test time per image elapsed {} s'.format(time_per_image/1000))

test time per image elapsed 2.0128020194734946 ms
test time per image elapsed 0.0020128020194734946 s


In [45]:
# deleting Surrogate 4
del Surrogate4
gc.collect()
torch.cuda.empty_cache()